In [2]:
# Here you can play with specific components of the system.

In [13]:
# Method for searching anime via AniList API
from src.anilist_query_searcher import search_anime

search_results = search_anime(
    # search_term="Attack on titan",
    # genres=["Sports", "Ecchi"],  # Using single genre parameter
    tags=["Volleyball"],
    per_page=5
)

# # Print all results first
print("All results:")
for anime in search_results:
    print(f"- {anime['title']['english'] or anime['title']['romaji']}")
    print(f"  Genres: {', '.join(anime['genres'])}")
    print(f"  Tags: {', '.join([tag['name'] for tag in anime['tags']])}")
    print(anime['coverImage'])
    print("-" * 60)

All results:
- HAIKYU!!
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Male Protagonist, Primarily Male Cast, Ensemble Cast, Shounen, Primarily Teen Cast, School, Coming of Age, Fitness, Cute Boys Doing Cute Things, Kuudere
{'medium': 'https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx20464-ooZUyBe4ptp9.png'}
------------------------------------------------------------
- HAIKYU!! 2nd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Ensemble Cast, Primarily Teen Cast, Primarily Male Cast, Shounen, Male Protagonist, Coming of Age, School, Tomboy
{'medium': 'https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx20992-aHgNbcalVEqk.png'}
------------------------------------------------------------
- HAIKYU!! 3rd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Shounen, Male Protagonist, Primarily Male Cast, Primarily Teen Cast, Ensemble Cast, School, Coming of Age
{'medium': 'https://s4.anilist.co/file/a

In [4]:
# Stage 1 - Request Parser
from crewai import Agent, Task, LLM
import os
from src.request_parser import AnimeSearchParams, OFFICIAL_GENRES

# Initialize LLM (using GPT-4o from OpenAI)
llm = LLM(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=2000,
    temperature=0
)

mapper = Agent(
    name="AniListRequestMapper",
    role="Filter extractor",
    goal="Return only the filters in the user's request, as minimal JSON.",
    backstory="An anime librarian who knows the difference between genres and tags.",
    allow_delegation=False,
    llm=llm
)

map_request = Task(
    description=(
        "USER_REQUEST:\n"
        "{user_request}\n\n"
        "Produce **one JSON object** that validates against AnimeSearchParams.\n"
        "Rules:\n"
        f"• Only items in this list may appear in `genres`: {OFFICIAL_GENRES}.\n"
        "• Any other descriptive phrase (moods, sub‑genres like 'School Life', "
        "adjectives like 'Wholesome') must go into `tags`.\n"
        "• Title‑case every word (e.g. 'school‑life' → 'School Life').\n"
        "Don't necessarily include genres unless they are specified in the request.\n"
        "• Omit every key whose value would be null.\n\n"
        "Example:\n"
        "USER_REQUEST:  Recommend a dark fantasy from 2020.\n"
        "OUTPUT: {\"genres\": [\"Fantasy\"], \"tags\": [\"Dark\"], \"year\": 2020}"
    ),
    expected_output="A JSON dict with only the mentioned filters, no nulls.",
    output_json=AnimeSearchParams,
    agent=mapper,
)

In [5]:
# Testing for stage 1 - Request Parser
from crewai import Crew, Process

prompt = "I want an isekai anime with some comedy"

crew = Crew(
    agents=[mapper],
    tasks=[map_request],
    process=Process.sequential,
)
crew.kickoff(inputs={"user_request": prompt})

params = {k: v for k, v in map_request.output.json_dict.items() if v is not None}
print(params)



{'genres': ['Comedy'], 'tags': ['Isekai']}


In [6]:
# Stage 2 - Anime searcher and recommender
from src.recommender import SearchAnimeTool
from crewai import Agent, Crew, Task

anime_tool = SearchAnimeTool()

anime_researcher = Agent(
    role="Anime Researcher",
    goal="Find animes that match a user query.",
    backstory="You are a seasoned anime critic.",
    tools=[anime_tool],
)

recommendation_task = Task(
    description="Using AniList, compile a list of five anime that match the query.",
    expected_output="Five recommendations with one‑sentence justifications.",
    agent=anime_researcher,
)

In [7]:
# Arming the whole crew: Stage 1 + Stage 2
crew = Crew(
    agents=[mapper, anime_researcher],
    tasks=[map_request, recommendation_task],
    process=Process.sequential,
)

In [6]:
# Testing the whole system

In [8]:
result = crew.kickoff(inputs={"user_request": "I want an isekai anime"})
print(result)

1. **Sword Art Online** - This pioneering series in the isekai genre features players trapped in a virtual reality game, blending intense action with emotional moments as they fight for their freedom. ![Sword Art Online](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx11757-SxYDUzdr9rh2.jpg)

2. **Re:ZERO -Starting Life in Another World-** - A gripping tale of time manipulation and psychological struggle, where the protagonist experiences death and resurrection, leading to harrowing choices that affect him and his companions. ![Re:ZERO](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx21355-wRVUrGxpvIQQ.jpg)

3. **No Game, No Life** - This vibrant and clever series follows a brother-sister duo who are transported to a world where games determine everything, showcasing their tactical brilliance and whimsical adventures. ![No Game, No Life](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/b19815-sEOQ9yQaPKlk.jpg)

4. **KONOSUBA -God's blessing 

In [10]:
result = crew.kickoff(inputs={"user_request": "I want a tennis anime with some comedy"})
print(result)

1. **The Prince of Tennis (Tennis no Ouji-sama)** - A classic sports anime that centers around the competitive world of tennis among high school students and showcases intense matches as well as comedic moments. ![The Prince of Tennis](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx22-yEguU9EmxkjK.png)

2. **Teekyuu** - A fast-paced comedy that humorously depicts the daily adventures of a high school tennis club, packed with gags and surreal moments that keep the energy high. ![Teekyuu](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx15125-2ngZyS4KrMqA.png)

3. **Teekyuu 2** - The sequel to Teekyuu continues the absurdities and outrageous antics of the same tennis club, maintaining its slice-of-life parody style with even more rapid-fire humor. ![Teekyuu 2](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx18121-k8UaPii7p9sl.jpg)

4. **Prince of Tennis II (Shin Tennis no Ouji-sama)** - This series revives the beloved characters from the or

In [14]:
result = crew.kickoff(inputs={"user_request": "I want an anime like ghost in the shell"})
print(result)

1. **PSYCHO-PASS** - This anime delves into a dystopian world where society is governed by an omnipresent law enforcement system, exploring the intersection of technology, morality, and psychology in a manner akin to *Ghost in the Shell*. ![Cover](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx13601-i42VFuHpqEOJ.jpg)

2. **Vivy: Fluorite Eye’s Song** - Combining action and poignant storytelling, it centers on an AI's existential journey through time, mirroring the philosophical themes of *Ghost in the Shell*. ![Cover](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx128546-UIwyhuhjxmL0.jpg)

3. **GHOST IN THE SHELL: Koukaku Kidoutai** - The original movie that set the standard for sci-fi anime, it combines stunning visuals and profound narrative, and remains essential viewing for fans of the genre. ![Cover](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx43-Y6EjeEMM14dj.png)

4. **PLUTO** - An adaptation of the *Astro Boy* arc, it feature